In [1]:
import pandas as pd
import time
from tqdm import tqdm
from uuid import uuid4

from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\vhqhr\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
INDEX_NAME = "funeral-services"
pc = Pinecone()
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print(index.describe_index_stats(), '\n\n')

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '289',
                                    'content-type': 'application/json',
                                    'date': 'Mon, 24 Nov 2025 00:43:22 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '143',
                                    'x-pinecone-request-id': '3124519123248885971',
                                    'x-pinecone-request-latency-ms': '142'}},
 'dimension': 1536,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'digital_legacy': {'vector_count': 68},
                'funeral_facilities': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'storageFullness': 0.0,
 'total_vector_count': 2824,
 'vector_type': '

In [ ]:
# index.delete(delete_all=True, namespace="funeral_facilities")
# print(index.describe_index_stats(), '\n\n')

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'funeral_facilities_with_gaslighting': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'total_vector_count': 2756,
 'vector_type': 'dense'} 




In [ ]:
# 각 파일별로 'type' 지정 및 컬럼 매핑 정의
file_configs = [
    {
        "filename": "cemeteries.csv",
        "type": "cemetery", 
        "kor_type" : "묘지",
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "colubarium.csv",
        "type": "columbarium", # 
        "kor_type" : "봉안당",
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "crematoriums.csv",
        "type": "crematorium", # 
        "kor_type" : "화장시설",
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "natural_burial.csv",
        "type": "natural_burial", # 
        "kor_type" : "자연장지",
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "funeral_homes.csv",
        "type": "funeral_home", # 
        "kor_type" : "장례식장",
        "mapping": {"소재지": "region", "장례식장 이름": "name", "주소": "address", "전화번호": "phone"}
        # 장례식장은 'location'(구/군) 컬럼이 없으므로 주소에서 추출 예정
    }
]

def load_and_unify_data(configs):
    combined_df = pd.DataFrame()
    
    def generate_text_logic(row):
        # 1. 기본 문장 구성
        content = f"{row['address']}에 위치한 {row['name']}입니다. 시설 유형은 {row['type']}입니다."
        name = str(row['name']) # 안전한 문자열 변환

        # 1. 천주교 (Catholic)
        catholic_keywords = [
            '천주교', '가톨릭', '성당', '성요셉', '성모', '수녀원', '프란치스꼬', '빈첸시오', '안젤로', '베다니아'
        ]
        
        # 2. 기독교/개신교 (Christian)
        christian_keywords = [
            '교회', '기독교', '크리스찬', '예수', '침례', '복음',
        ]
        
        # 3. 불교 (Buddhist)
        buddhist_keywords = [
            '불교', '조계종', '태고종', '연화', '극락', '지장', '미타', '관음', '만불', '영각당'
        ]
        
        # 4. 공설/시립 (Public)
        public_keywords = ['시립', '군립', '공설', '시설관리공단']
        
        # 5. 사설/재단 (Foundation/Private)
        foundation_keywords = ['(재)', '재단법인', '(주)', '(사)']

        # 천주교 체크
        if any(k in name for k in catholic_keywords):
            content += " 이 곳은 천주교(가톨릭)와 관련이 있는 시설입니다."
            
        # 기독교 체크
        elif any(k in name for k in christian_keywords):
            content += " 이 곳은 기독교(개신교)와 관련이 있는 시설입니다."
            
        # 불교 체크 (이름이 '사'로 끝나거나 키워드 포함)
        elif any(k in name for k in buddhist_keywords): 
                content += " 이 곳은 불교와 관련이 있는 시설입니다."

        # 공설/사설 체크 (종교 시설이 아닌 경우에만 주로 설명하거나, 추가 정보로 제공)
        if any(k in name for k in public_keywords):
            content += " 지방자치단체 운영, 공설(시립/군립)과 관련이 있는 시설입니다."
        elif any(k in name for k in foundation_keywords):
            content += " 사설 시설과 관련이 있는 시설입니다."

        return content

    for config in configs:
        try:
            # 파일 읽기 (인코딩 에러 처리)
            try:
                df = pd.read_csv('../data/processed/' + config["filename"])
            except UnicodeDecodeError:
                df = pd.read_csv('../data/processed/' + config["filename"], encoding='cp949')
            
            if config["filename"] == "colubarium.csv":
                df.loc[df['시설명'] == "군산시추모관", '주소'] = '전북 군산시 임피면 보석리 401-5'

            # 컬럼명 변경
            df = df.rename(columns=config["mapping"])
            # type 컬럼 추가
            df['type'] = config["kor_type"]
            
            
            # region: 주소 앞 2개 단어 추출 (예: "경기도 수원시")
            df['region'] = df['address'].apply(
                lambda x: " ".join(x.split()[:2]) if isinstance(x, str) else ""
            )
            
            # 필요한 컬럼만 선택 및 결측치 처리
            target_cols = ['type', 'region', 'name', 'address', 'phone']
            
            # 데이터프레임에 없는 컬럼이 있다면 빈 값으로 생성
            for col in target_cols:
                if col not in df.columns:
                    df[col] = ""
            
            df = df[target_cols].fillna("정보없음")
            
            # 검색에 사용될 'text' 필드 생성 (임베딩 대상)
            df['text'] = df.apply(generate_text_logic, axis=1)
            
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"[OK] {config['filename']} 로드 완료 ({len(df)}건)")
            
        except Exception as e:
            print(f"[Error] {config['filename']} 처리 중 오류: {e}")
            
    return combined_df

# ---------------------------------------------------------
# 3. Pinecone 적재 로직
# ---------------------------------------------------------
def upload_to_pinecone(df):
    # Pinecone 초기화
    pc = Pinecone()
    
    # 인덱스 확인 및 생성 (없을 경우)
    if INDEX_NAME not in pc.list_indexes().names():
        pc.create_index(
            name=INDEX_NAME,
            dimension=1536, # OpenAI text-embedding-3-small 기준
            metric='cosine',
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
    
    batch_size = 100
    total_rows = len(df)
    
    print(f"\n총 {total_rows}개의 데이터를 Pinecone에 적재합니다...")
    
    for i in tqdm(range(0, total_rows, batch_size)):
        batch = df.iloc[i:i+batch_size]
        
        # 1. 텍스트 임베딩 생성
        texts = batch['text'].tolist()
        ids = [str(uuid4()) for _ in range(len(batch))]
        embeds = embeddings.embed_documents(texts)
        
        # 2. 메타데이터 준비 (full_region 포함)
        # Pinecone은 메타데이터 값을 문자열, 숫자, 불리언, 리스트만 허용함
        metadatas = batch[['type', 'region', 'name', 'address', 'phone']].to_dict(orient='records')
        
        # text 필드도 메타데이터에 포함하여 RAG 시 원문 참조 가능하게 함
        for j, meta in enumerate(metadatas):
            meta['text'] = texts[j]
            
        # 3. 벡터 데이터 구성 (id, vector, metadata)
        vectors = list(zip(ids, embeds, metadatas))
        
        # 4. 업로드
        index.upsert(vectors=vectors, namespace='funeral_facilities')
        
    print("적재 완료!")

# ---------------------------------------------------------
# 실행
# ---------------------------------------------------------
if __name__ == "__main__":
    # 1. 데이터 로드 및 전처리
    unified_df = load_and_unify_data(file_configs)
    
    # 2. 데이터 확인
    print("\n--- 데이터 예시 ---")
    # print(unified_df.head(2))
    display(unified_df)
    # 3. Pinecone 적재 (API 키 설정 후 주석 해제하여 실행)
    upload_to_pinecone(unified_df)

[OK] cemeteries.csv 로드 완료 (524건)
[OK] colubarium.csv 로드 완료 (626건)
[OK] crematoriums.csv 로드 완료 (62건)
[OK] natural_burial.csv 로드 완료 (219건)
[OK] funeral_homes.csv 로드 완료 (1087건)

--- 데이터 예시 ---


,type,region,name,address,phone,text
0,묘지,전라북도 완주군,(재)호정공원(묘지),전라북도 완주군 화산면 운곡로 381-64 (운곡리),063-214-1009,전라북도 완주군 화산면 운곡로 381-64 (운곡리)에 위치한 (재)호정공원(묘지)...
1,묘지,경상북도 경주시,(재)대명공원묘원,경상북도 경주시 강동면 비학로 50-126 (단구리),054-762-3810,경상북도 경주시 강동면 비학로 50-126 (단구리)에 위치한 (재)대명공원묘원입니...
2,묘지,경기도 광주시,(재)시안 가족추모공원(매장묘),경기도 광주시 오포안로 17 (문형동),031-766-2894,경기도 광주시 오포안로 17 (문형동)에 위치한 (재)시안 가족추모공원(매장묘)입니...
3,묘지,전라북도 정읍시,정읍시입암묘지공원,전라북도 정읍시 입암면 연월리 산159,063-539-7153,전라북도 정읍시 입암면 연월리 산159에 위치한 정읍시입암묘지공원입니다. 시설 유형...
4,묘지,경상북도 김천시,천주교 김천공원묘원,경상북도 김천시 개령면 황계길 280-41 (황계리),054-433-3880,경상북도 김천시 개령면 황계길 280-41 (황계리)에 위치한 천주교 김천공원묘원입...
...,...,...,...,...,...,...
2513,장례식장,경상남도 거창군,거창적십자병원장례식장,"경상남도 거창군 거창읍 중앙로 91 (상림리, 거창적십자병원)",055-944-4482,"경상남도 거창군 거창읍 중앙로 91 (상림리, 거창적십자병원)에 위치한 거창적십자병..."
2514,장례식장,경상남도 거창군,거창서경병원장례식장( 거창한국병원장례식장),"경상남도 거창군 거창읍 송정1길 24-13 (송정리, 서경병원)",055-945-0130,"경상남도 거창군 거창읍 송정1길 24-13 (송정리, 서경병원)에 위치한 거창서경병..."
2515,장례식장,경상남도 합천군,합천추모공원장례식장,경상남도 합천군 합천읍 합천호수로 1613 (합천리),055-934-4444,경상남도 합천군 합천읍 합천호수로 1613 (합천리)에 위치한 합천추모공원장례식장입...
2516,장례식장,경상남도 합천군,합천고려병원 장례식장,경상남도 합천군 대양면 대야로 737-20 (정양리),055-931-4464,경상남도 합천군 대양면 대야로 737-20 (정양리)에 위치한 합천고려병원 장례식장...



총 2518개의 데이터를 Pinecone에 적재합니다...


100%|██████████| 26/26 [00:59<00:00,  2.28s/it]

적재 완료!


In [57]:
import json

def extract_facilities_region_list(df):
    """type별 지역 리스트 추출"""
    region_dict = {}
    
    for facility_type in df['type'].unique():
        # full_region만 추출하고 정렬
        regions = sorted(df[df['type'] == facility_type]['region'].dropna().unique())
        region_dict[facility_type] = regions
    
    return region_dict

# 실행
facilities_regions = extract_facilities_region_list(unified_df)

# JSON 저장
with open('./data/processed/facilities_region_list.json', 'w', encoding='utf-8') as f:
    json.dump(facilities_regions, f, ensure_ascii=False, indent=2)

print("facilities_region_list.json 생성 완료!")
print(json.dumps(facilities_regions, ensure_ascii=False, indent=2))

facilities_region_list.json 생성 완료!
{
  "묘지": [
    "강원도 태백시",
    "강원특별자치도 강릉시",
    "강원특별자치도 고성군",
    "강원특별자치도 동해시",
    "강원특별자치도 삼척시",
    "강원특별자치도 속초시",
    "강원특별자치도 양구군",
    "강원특별자치도 양양군",
    "강원특별자치도 영월군",
    "강원특별자치도 원주시",
    "강원특별자치도 인제군",
    "강원특별자치도 정선군",
    "강원특별자치도 철원군",
    "강원특별자치도 춘천시",
    "강원특별자치도 평창군",
    "강원특별자치도 홍천군",
    "강원특별자치도 화천군",
    "강원특별자치도 횡성군",
    "경기 파주시",
    "경기도 가평군",
    "경기도 고양시",
    "경기도 광주시",
    "경기도 구리시",
    "경기도 김포시",
    "경기도 남양주시",
    "경기도 동두천시",
    "경기도 성남시",
    "경기도 시흥시",
    "경기도 안산시",
    "경기도 안성시",
    "경기도 양주시",
    "경기도 양평군",
    "경기도 여주시",
    "경기도 연천군",
    "경기도 오산시",
    "경기도 용인시",
    "경기도 의왕시",
    "경기도 의정부시",
    "경기도 이천시",
    "경기도 파주시",
    "경기도 평택시",
    "경기도 포천시",
    "경기도 화성시",
    "경상남도 거제시",
    "경상남도 거창군",
    "경상남도 고성군",
    "경상남도 김해시",
    "경상남도 남해군",
    "경상남도 사천시",
    "경상남도 양산시",
    "경상남도 의령군",
    "경상남도 진주시",
    "경상남도 창원시",
    "경상남도 함안군",
    "경상남도 함양군",
    "경상북도 경산시",
    "경상북도 경주시",
    "경상북도 구미시"

In [ ]:
# for i, row in unified_df.iterrows():
#     print(" ".join(row.address.split()[:2]))

전라북도 완주군
경상북도 경주시
경기도 광주시
전라북도 정읍시
경상북도 김천시
경상남도 양산시
경기도 여주시
인천광역시 서구
제주특별자치도 서귀포시
경기도 성남시
제주특별자치도 서귀포시
대구광역시 북구
인천광역시 옹진군
인천광역시 강화군
경기도 포천시
전라남도 나주시
인천광역시 강화군
경상남도 김해시
경기도 가평군
경기도 포천시
경기도 고양시
충청남도 당진시
강원특별자치도 춘천시
경기도 파주시
경기도 양주시
경기도 양주시
전라남도 영암군
경상남도 거창군
경기도 포천시
경기도 양주시
경상북도 성주군
전라남도 여수시
대전광역시 서구
경상남도 창원시
경기도 의왕시
제주특별자치도 제주시
인천광역시 강화군
인천광역시 강화군
인천광역시 강화군
광주광역시 남구
경기도 김포시
전라북도 전주시
경기도 파주시
경기도 포천시
인천광역시 강화군
대구광역시 군위군
인천광역시 강화군
광주광역시 북구
전라남도 나주시
강원특별자치도 철원군
전라북도 고창군
강원특별자치도 인제군
경기도 파주시
인천광역시 강화군
강원특별자치도 양양군
경기도 용인시
경기도 남양주시
부산광역시 기장군
경상남도 창원시
경상남도 거창군
경기도 안산시
경상북도 김천시
경기도 포천시
강원특별자치도 강릉시
제주특별자치도 제주시
경기도 파주시
충청남도 홍성군
충청북도 충주시
경기도 파주시
전라남도 완도군
경기도 파주시
인천광역시 강화군
경상북도 칠곡군
경상북도 구미시
제주특별자치도 제주시
충청북도 옥천군
강원특별자치도 고성군
인천광역시 강화군
전라북도 김제시
경기도 파주시
부산광역시 금정구
충청북도 충주시
충청남도 예산군
전라북도 장수군
전라북도 익산시
부산광역시 기장군
제주특별자치도 서귀포시
제주특별자치도 서귀포시
경기도 광주시
경기도 용인시
충청남도 논산시
전라남도 나주시
경기도 양주시
충청남도 천안시
강원특별자치도 영월군
인천광역시 강화군
충청남도 부여군
경기도 파주시
경상북도 성주군
경기도 의왕시
인천광역시 강화군
인천광역시 옹진군
전라북도 고창군
경기도 양주시
강원특별자치도 고성군
경기도 여주시
경기도 포천시
전라

In [58]:
len(unified_df[unified_df['phone'] == "정보없음"])


17

In [33]:
unified_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     2518 non-null   object
 1   region   2518 non-null   object
 2   name     2518 non-null   object
 3   address  2518 non-null   object
 4   phone    2518 non-null   object
 5   text     2518 non-null   object
dtypes: object(6)
memory usage: 118.2+ KB


In [50]:
df = pd.read_csv('./data/processed/colubarium.csv')
print(df.info())
print(df[df['시설명']=='군산시추모관'])
df.loc[df['시설명'] == "군산시추모관", '주소'] = '전북 군산시 임피면 보석리 401-5'
print(df[df['시설명']=='군산시추모관'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      626 non-null    object
 1   소재지     626 non-null    object
 2   시설명     626 non-null    object
 3   주소      625 non-null    object
 4   연락처     617 non-null    object
dtypes: object(5)
memory usage: 24.6+ KB
None
          지역  소재지     시설명   주소           연락처
348  전북특별자치도  군산시  군산시추모관  NaN  063-454-7952
          지역  소재지     시설명                    주소           연락처
348  전북특별자치도  군산시  군산시추모관  전북 군산시 임피면 보석리 401-5  063-454-7952
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      626 non-null    object
 1   소재지     626 non-null    object
 2   시설명     626 non-null    object
 3   주소      626 non-null    object
 4   연락처     617 non-null    object
dtypes: object(5)
memory usa